# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [9]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [10]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [11]:
import numpy as np
np.random.seed(10)
ARB = np.random.randint(5,50,5)
PRO = np.random.randint(2,10,5)
ARB, PRO

(array([14, 41, 20,  5, 33]), array([3, 7, 2, 7, 3]))

In [50]:
POINTS = []
for i in range (len(PRO)):
    modelo, f1 = Bosque.RegresionBosque(X, y, ARB[i], PRO[i])
    POINTS.append(f1)
POINTS
    

[0.9555555555555556,
 0.9555555555555556,
 0.9333333333333333,
 0.9333333333333333,
 0.9444444444444444]

In [68]:
ARB_vect = ARB.reshape([-1,1])
PRO_vect = PRO.reshape ([-1,1])
HPM = np.hstack((ARB_vect,PRO_vect))
HPM

array([[14,  3],
       [41,  7],
       [20,  2],
       [ 5,  7],
       [33,  3]])

In [71]:
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0*RBF(length_scale = 1)
xA = np.linspace(0,50,5).reshape([-1,1])
xP = np.linspace(2,10,5).reshape([-1,1])
MAP = np.hstack((xA,xP))
gp = GPR(kernel=kernel, n_restarts_optimizer=10).fit(MAP, POINTS)
y_pred, y_std = gp.predict(MAP, return_std=True)
y_pred_high = y_pred + 1.96*y_std
i_next = np.argmax(y_pred_high)
new_X= np.vstack((HPM,HPM[i_next]))
new_X

array([[14,  3],
       [41,  7],
       [20,  2],
       [ 5,  7],
       [33,  3],
       [14,  3]])

In [72]:
HPM[i_next]

array([14,  3])

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.